# 数据裁剪

In [48]:
import numpy as np
import scipy as sp
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import xarray as xr
import shapely
import rioxarray as rxr
import os
import rasterio

暂时使用框选法对数据进行初步处理

框选范围选择为$32^\circ N - 40^\circ N$ ， $114^\circ E - 121^\circ E$ 

由于数据量较大，这里使用数据规模最大的CLCD数据作为测试数据集

> 刚经过测试CLCD太大了跑不起来
> 上温度试试吧

In [49]:
cutBox = [114,32,121,40]
cutFrame = shapely.geometry.box(*cutBox)

cutGdf = gpd.GeoDataFrame({'geometry':cutFrame},index=[0],crs={'init':'epsg:4326'})
cutGdf.explore()

/home/zyzh0/anaconda3/envs/gis/lib/python3.9/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


打开TEMP的第一个文件，输出metadata

In [50]:
tempFrame = xr.open_dataset(r"datas/TEMP/extracted/Temp_pred_2000_1_Tmean.tif").squeeze("band",drop=True)
tempFrame


<xarray.Dataset>
Dimensions:      (x: 6856, y: 4319)
Coordinates:
  * x            (x) float64 73.5 73.51 73.52 73.53 ... 135.1 135.1 135.1 135.1
  * y            (y) float64 53.56 53.55 53.54 53.54 ... 14.8 14.79 14.78 14.77
    spatial_ref  int64 ...
Data variables:
    band_data    (y, x) float32 ...

cutting
不要画图！！！！

In [51]:
tempFrameMasked = tempFrame.rio.clip(cutGdf.geometry,crs=tempFrame.rio.crs)

#tempFrameMasked.rio.to_raster(r"datas/TEMP/Temp_pred_2000_1_Tmean_cut.tif")

In [52]:
def cutFrame(frame,box):
    cutFrame = shapely.geometry.box(*box)
    cutGdf = gpd.GeoDataFrame({'geometry':cutFrame},index=[0],crs={'init':'epsg:4326'})
    cutFrameMasked = frame.rio.clip(cutGdf.geometry,crs=frame.rio.crs)
    return cutFrameMasked

pathPrefix = r"datas/TEMP/extracted/"

pathLst = []

for i in range(2000,2016):
    for j in range(1,13):
        pathLst.append(pathPrefix+"Temp_pred_"+str(i)+"_"+str(j)+"_Tmean.tif")

pathLst

['datas/TEMP/extracted/Temp_pred_2000_1_Tmean.tif',
 'datas/TEMP/extracted/Temp_pred_2000_2_Tmean.tif',
 'datas/TEMP/extracted/Temp_pred_2000_3_Tmean.tif',
 'datas/TEMP/extracted/Temp_pred_2000_4_Tmean.tif',
 'datas/TEMP/extracted/Temp_pred_2000_5_Tmean.tif',
 'datas/TEMP/extracted/Temp_pred_2000_6_Tmean.tif',
 'datas/TEMP/extracted/Temp_pred_2000_7_Tmean.tif',
 'datas/TEMP/extracted/Temp_pred_2000_8_Tmean.tif',
 'datas/TEMP/extracted/Temp_pred_2000_9_Tmean.tif',
 'datas/TEMP/extracted/Temp_pred_2000_10_Tmean.tif',
 'datas/TEMP/extracted/Temp_pred_2000_11_Tmean.tif',
 'datas/TEMP/extracted/Temp_pred_2000_12_Tmean.tif',
 'datas/TEMP/extracted/Temp_pred_2001_1_Tmean.tif',
 'datas/TEMP/extracted/Temp_pred_2001_2_Tmean.tif',
 'datas/TEMP/extracted/Temp_pred_2001_3_Tmean.tif',
 'datas/TEMP/extracted/Temp_pred_2001_4_Tmean.tif',
 'datas/TEMP/extracted/Temp_pred_2001_5_Tmean.tif',
 'datas/TEMP/extracted/Temp_pred_2001_6_Tmean.tif',
 'datas/TEMP/extracted/Temp_pred_2001_7_Tmean.tif',
 'datas/T

In [53]:
outputDir = r"outputs/temp-cutted/"

for i in pathLst:
    tempFrame = xr.open_dataset(i).squeeze("band",drop=True)
    tempFrameMasked = cutFrame(tempFrame,cutBox)
    tempFrameMasked.rio.to_raster(outputDir+i[-17:-4]+"_cut.tif")

/home/zyzh0/anaconda3/envs/gis/lib/python3.9/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/zyzh0/anaconda3/envs/gis/lib/python3.9/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/zyzh0/anaconda3/envs/gis/lib/python3.9/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred init